In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [2]:
# Import our input dataset
crime_df = pd.read_csv('All_data_cleaned.csv')
crime_df.head()

crime_df.dtypes

Year                                                            int64
City                                                           object
Population 25 years and over - Less than 9th grade              int64
Percent Population 25 years and over - Less than 9th grade    float64
Population 25 years and over - 9th to12th (No Diploma)          int64
                                                               ...   
Percent 15 to 24                                              float64
Percent Uneducated                                            float64
Percent Higher Education                                      float64
Sum Uneducated                                                  int64
Sum Higher Education                                            int64
Length: 61, dtype: object

In [3]:
# One-hot encode the 'City' column
crime_encoded = pd.get_dummies(crime_df, columns=['City'])

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'City' column
crime_df['City_encoded'] = label_encoder.fit_transform(crime_df['City'])

crime_df

,Year,City,Population 25 years and over - Less than 9th grade,Percent Population 25 years and over - Less than 9th grade,Population 25 years and over - 9th to12th (No Diploma),Percent Population 25 years and over - 9th to12th (No Diploma),Population 25 years and over - High School Graduate (and equivalent),Percent Population 25 years and over - High School Graduate (and equivalent),"Population 25 years and over - Some college, no degree","Percent Population 25 years and over - Some college, no degree",...,Percent Home Occupied,Percent Renter Occupied,Percent 25+,Percent 14-,Percent 15 to 24,Percent Uneducated,Percent Higher Education,Sum Uneducated,Sum Higher Education,City_encoded
0,2010,Alameda,2752,5.50,2234,4.50,8546,17.00,10009,19.90,...,46.920853,53.079147,65.8,19.8,12.3,10.00,53.10,4986,26639,0
1,2010,Alhambra,6236,10.10,4575,7.40,13358,21.70,12091,19.60,...,38.053381,61.946619,71.1,12.9,13.1,17.50,41.10,10811,25307,1
2,2010,Anaheim,32561,15.50,22329,10.70,50982,24.30,39407,18.80,...,48.388510,51.611490,61.0,21.2,16.7,26.20,30.70,54890,64362,2
3,2010,Antioch,4874,7.70,4633,7.30,17092,26.90,20113,31.70,...,62.767194,37.232806,60.7,22.9,15.5,15.00,26.50,9507,16815,3
4,2010,Apple Valley,639,1.50,3353,7.70,14524,33.40,11690,26.90,...,70.690377,29.309623,60.4,23.4,13.9,9.20,30.50,3992,13303,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,2019,Vista,10080,15.05,6638,9.91,13245,19.77,16027,23.93,...,45.724859,54.275141,64.4,19.9,14.1,24.96,31.34,16718,20990,129
1161,2019,West Covina,5308,7.09,4530,6.05,20919,27.95,16083,21.49,...,57.975360,42.024640,69.5,16.9,11.9,13.14,37.41,9838,28003,131
1162,2019,Westminster,8307,13.03,6607,10.36,15641,24.53,11273,17.68,...,47.216869,52.783131,68.0,18.2,11.5,23.39,34.41,14914,21943,132
1163,2019,Whittier,3021,5.44,3921,7.06,16232,29.24,12079,21.76,...,60.928433,39.071567,63.3,21.0,13.9,12.50,36.49,6942,20258,133


In [4]:
# Fix the seed for random number generation
seed_value = 8
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Remove crime and city target from features data
X = crime_df.copy()
X.drop(columns=["Crime_Rate_per_100k", 
                "City",
                "Violent Crimes Sum"
               ], axis=1, inplace=True)
y = crime_df['Crime_Rate_per_100k']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=8)

In [5]:
# build model
model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model1.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model1.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1)

# evaluate the model
test_loss = model1.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model1.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 231015.3281 - val_loss: 193008.4062
Epoch 2/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 143409.6875 - val_loss: 64024.4805
Epoch 3/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 81975.3047 - val_loss: 52809.5469
Epoch 4/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 70064.7188 - val_loss: 45064.1406
Epoch 5/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 61946.3398 - val_loss: 39711.3516
Epoch 6/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 54619.5859 - val_loss: 36047.3047
Epoch 7/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50032.5156 - val_loss: 33673.8555
Epoch 8/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 46296.9805 - val_loss: 32054.2832
Epoch 9/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43391.8867 - val_loss: 30760.0957
Epoch 10/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40829.1172 - val_loss: 29443.7480
Epoch 11/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 38162.0391 - val_l

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1698.4559 - val_loss: 24585.2871
Epoch 66/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1640.3622 - val_loss: 24788.5293
Epoch 67/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1582.7316 - val_loss: 24930.2930
Epoch 68/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1496.8634 - val_loss: 25281.5898
Epoch 69/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1444.9296 - val_loss: 25050.6738
Epoch 70/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1366.0781 - val_loss: 25559.7207
Epoch 71/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1314.8777 - val_loss: 25616.2109
Epoch 72/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1251.8976 - val_loss: 25796.4746
Epoch 73/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1214.4374 - val_loss: 25939.8574
Epoch 74/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1154.1183 - val_loss: 26167.8047
Epoch 75/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1117.2233 - val_loss: 2

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 489.2573 - val_loss: 27628.0469
Epoch 130/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 487.8701 - val_loss: 28095.1875
Epoch 131/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 498.2907 - val_loss: 28707.3359
Epoch 132/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 516.5629 - val_loss: 29231.2852
Epoch 133/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 536.1060 - val_loss: 30026.6016
Epoch 134/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 574.2560 - val_loss: 30555.7988
Epoch 135/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 637.4235 - val_loss: 31500.6875
Epoch 136/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 732.7083 - val_loss: 32333.9570
Epoch 137/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 910.1108 - val_loss: 33305.6406
Epoch 138/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1071.1190 - val_loss: 33962.1172
Epoch 139/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1252.2488 - val_loss: 

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 639.8950 - val_loss: 25967.0273
Epoch 194/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 663.6545 - val_loss: 24872.9551
Epoch 195/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 742.7161 - val_loss: 23969.2871
Epoch 196/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 894.6768 - val_loss: 23518.8262
Epoch 197/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1196.3171 - val_loss: 23498.1152
Epoch 198/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1537.2570 - val_loss: 23973.0312
Epoch 199/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1971.2090 - val_loss: 23962.9609
Epoch 200/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2344.6128 - val_loss: 26393.8750
Epoch 201/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2182.1296 - val_loss: 32589.4141
Epoch 202/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2943.3203 - val_loss: 30142.8164
Epoch 203/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2974.4338 - val_l

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1279.0386 - val_loss: 28709.5020
Epoch 258/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1114.1730 - val_loss: 26678.6016
Epoch 259/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1143.9739 - val_loss: 26298.2383
Epoch 260/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1069.6553 - val_loss: 28208.3594
Epoch 261/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 874.3443 - val_loss: 30681.5234
Epoch 262/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1063.7037 - val_loss: 30212.2969
Epoch 263/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1357.3167 - val_loss: 27535.2832
Epoch 264/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1285.8230 - val_loss: 25920.9941
Epoch 265/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1287.3264 - val_loss: 25963.8145
Epoch 266/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1186.8011 - val_loss: 27957.4961
Epoch 267/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 995.5767 - val

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8068.5293 - val_loss: 44522.8984
Epoch 322/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9476.4551 - val_loss: 42713.2812
Epoch 323/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10349.1094 - val_loss: 39461.3867
Epoch 324/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10325.2529 - val_loss: 34119.6758
Epoch 325/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9575.9434 - val_loss: 27887.4883
Epoch 326/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7878.4268 - val_loss: 23378.7676
Epoch 327/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6704.6963 - val_loss: 22723.5000
Epoch 328/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5488.8467 - val_loss: 24178.2871
Epoch 329/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4789.0186 - val_loss: 25704.4336
Epoch 330/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4313.3184 - val_loss: 26052.5527
Epoch 331/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3499.0654 -

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 689.9175 - val_loss: 24283.0605
Epoch 386/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 690.2576 - val_loss: 24065.5430
Epoch 387/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 660.1246 - val_loss: 24202.8652
Epoch 388/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 675.8547 - val_loss: 23896.0469
Epoch 389/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 652.4625 - val_loss: 23605.8223
Epoch 390/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 658.8622 - val_loss: 23343.8633
Epoch 391/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 637.2833 - val_loss: 23035.5840
Epoch 392/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 614.8383 - val_loss: 22944.9766
Epoch 393/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 674.7887 - val_loss: 22601.0840
Epoch 394/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 706.1005 - val_loss: 22390.7168
Epoch 395/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 814.1974 - val_loss: 22

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 278.3722 - val_loss: 24671.9531
Epoch 450/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 238.6248 - val_loss: 24800.0117
Epoch 451/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 256.0565 - val_loss: 25296.5352
Epoch 452/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 346.4002 - val_loss: 25992.4688
Epoch 453/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 470.6175 - val_loss: 26619.1270
Epoch 454/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 578.4667 - val_loss: 26910.2383
Epoch 455/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 640.2783 - val_loss: 26732.2578
Epoch 456/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 627.2170 - val_loss: 26314.7520
Epoch 457/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 539.6427 - val_loss: 25671.2832
Epoch 458/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 438.7861 - val_loss: 25066.4805
Epoch 459/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 370.0989 - val_loss: 24

In [6]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    202.156784  119.793729
1    310.339874  359.563884
2    235.825073  168.934525
3    341.281647  355.505368
4    464.882263  433.151080
..          ...         ...
112  643.369324  487.957347
113  309.545837  216.123044
114  397.738434  425.953734
115  363.299835  436.229287
116  435.250549  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 76.96630766293968
Mean Squared Error (MSE): 11794.694722134414
Root Mean Squared Error (RMSE): 108.60338264591216
R-squared (R²): 0.7912020941080277
Mean Absolute Error: 76.97 degrees.
Accuracy: 69.67 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [7]:
# build model
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model2.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model2.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1)

# evaluate the model
test_loss = model2.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model2.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 229223.3125 - val_loss: 157769.8438
Epoch 2/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 123156.8984 - val_loss: 67102.4062
Epoch 3/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 81088.1094 - val_loss: 52434.5117
Epoch 4/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 67031.8750 - val_loss: 44298.3906
Epoch 5/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 58911.5039 - val_loss: 39188.2383
Epoch 6/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 52103.5625 - val_loss: 36337.6562
Epoch 7/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 48305.7070 - val_loss: 34128.2188
Epoch 8/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 45021.3867 - val_loss: 32574.2129
Epoch 9/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 42348.2461 - val_loss: 31076.9336
Epoch 10/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39645.9727 - val_loss: 29662.1582
Epoch 11/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36713.2383 - val_l

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1983.1071 - val_loss: 24055.6172
Epoch 66/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1842.0098 - val_loss: 24572.9043
Epoch 67/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1737.7415 - val_loss: 25242.5312
Epoch 68/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1628.3087 - val_loss: 25645.6191
Epoch 69/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1480.7878 - val_loss: 25789.1875
Epoch 70/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1356.5876 - val_loss: 25593.9453
Epoch 71/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1257.2513 - val_loss: 25402.3809
Epoch 72/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1168.8496 - val_loss: 25151.0000
Epoch 73/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1112.3319 - val_loss: 24995.6289
Epoch 74/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1046.4034 - val_loss: 24945.9629
Epoch 75/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1008.8395 - val_loss: 2

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1002.4489 - val_loss: 36980.9297
Epoch 130/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1039.4661 - val_loss: 38504.0820
Epoch 131/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1098.8962 - val_loss: 39553.1484
Epoch 132/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1123.7234 - val_loss: 40911.0430
Epoch 133/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1175.1791 - val_loss: 42691.0000
Epoch 134/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1330.9913 - val_loss: 43509.9023
Epoch 135/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1454.8311 - val_loss: 44318.9609
Epoch 136/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1591.2551 - val_loss: 44289.9023
Epoch 137/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1757.9263 - val_loss: 43893.6328
Epoch 138/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2081.7842 - val_loss: 41967.5469
Epoch 139/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2450.3135 - v

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2631.8560 - val_loss: 28507.7109
Epoch 194/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1686.5906 - val_loss: 35816.4805
Epoch 195/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1843.4656 - val_loss: 34188.1094
Epoch 196/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2261.9241 - val_loss: 28453.2812
Epoch 197/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1446.9243 - val_loss: 25738.0879
Epoch 198/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1006.4782 - val_loss: 26250.8633
Epoch 199/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 794.0574 - val_loss: 28658.5664
Epoch 200/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 861.9247 - val_loss: 30983.7598
Epoch 201/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1178.5327 - val_loss: 30618.8340
Epoch 202/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1367.0120 - val_loss: 28289.6348
Epoch 203/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1240.7050 - val

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1436.9540 - val_loss: 24737.2891
Epoch 258/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1599.1532 - val_loss: 24534.4570
Epoch 259/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1354.7057 - val_loss: 25625.4102
Epoch 260/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1020.1659 - val_loss: 28602.8418
Epoch 261/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1499.2919 - val_loss: 27557.3242
Epoch 262/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1495.9740 - val_loss: 25068.0352
Epoch 263/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1325.5269 - val_loss: 24176.6133
Epoch 264/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1392.9681 - val_loss: 25210.3457
Epoch 265/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1351.9454 - val_loss: 27363.6953
Epoch 266/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1195.7344 - val_loss: 28168.5488
Epoch 267/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1421.3489 - v

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1377.2574 - val_loss: 23150.8789
Epoch 322/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1179.5541 - val_loss: 22302.2246
Epoch 323/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 961.0443 - val_loss: 21361.7598
Epoch 324/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 738.6429 - val_loss: 21741.3438
Epoch 325/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 784.9958 - val_loss: 22972.1055
Epoch 326/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 783.7130 - val_loss: 24611.2031
Epoch 327/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1054.9030 - val_loss: 23801.6250
Epoch 328/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1119.1536 - val_loss: 22626.8008
Epoch 329/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 959.5644 - val_loss: 22125.1152
Epoch 330/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 987.8620 - val_loss: 22786.0703
Epoch 331/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1003.0876 - val_los

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 117.8384 - val_loss: 24068.0312
Epoch 386/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 107.0945 - val_loss: 24109.4961
Epoch 387/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 96.7627 - val_loss: 24077.2793
Epoch 388/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 86.9765 - val_loss: 24114.2070
Epoch 389/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 79.4833 - val_loss: 24159.3848
Epoch 390/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 71.8813 - val_loss: 24189.5254
Epoch 391/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 65.9828 - val_loss: 24233.6738
Epoch 392/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 60.6311 - val_loss: 24279.1738
Epoch 393/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 56.3878 - val_loss: 24351.5879
Epoch 394/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53.0644 - val_loss: 24417.1582
Epoch 395/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50.6517 - val_loss: 24506.3848


Epoch 450/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1086.0262 - val_loss: 27420.5996
Epoch 451/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 839.5190 - val_loss: 30556.3496
Epoch 452/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1196.0627 - val_loss: 27092.1387
Epoch 453/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 720.1148 - val_loss: 26440.0078
Epoch 454/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 400.8024 - val_loss: 28822.2227
Epoch 455/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 588.0262 - val_loss: 26965.9473
Epoch 456/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 648.0458 - val_loss: 26067.0762
Epoch 457/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 569.7667 - val_loss: 27257.8164
Epoch 458/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 537.4374 - val_loss: 26733.7539
Epoch 459/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 489.0234 - val_loss: 25511.3047
Epoch 460/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 599.438

In [8]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    119.132866  119.793729
1    543.474854  359.563884
2    155.504318  168.934525
3    345.068268  355.505368
4    426.082001  433.151080
..          ...         ...
112  762.242249  487.957347
113  358.826233  216.123044
114  307.910645  425.953734
115  392.775299  436.229287
116  344.986725  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 89.62743275012365
Mean Squared Error (MSE): 15956.040101248604
Root Mean Squared Error (RMSE): 126.317220129516
R-squared (R²): 0.7175350580954973
Mean Absolute Error: 89.63 degrees.
Accuracy: 69.73 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [27]:
# build model
model7 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model7.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model7.fit(X_train, y_train, epochs=300, batch_size=32, validation_split=0.1)

# evaluate the model
test_loss = model7.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model7.predict(X_test)

Epoch 1/300


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 228904.7656 - val_loss: 158315.6250
Epoch 2/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 121232.2656 - val_loss: 65532.6797
Epoch 3/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 78644.0234 - val_loss: 51646.3711
Epoch 4/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 65649.0156 - val_loss: 44019.5039
Epoch 5/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 57274.7812 - val_loss: 39110.5938
Epoch 6/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51202.1250 - val_loss: 36353.3164
Epoch 7/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 47391.7461 - val_loss: 34365.9961
Epoch 8/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 44149.5469 - val_loss: 32574.6758
Epoch 9/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 41137.7617 - val_loss: 30986.7793
Epoch 10/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38331.1367 - val_loss: 29675.8164
Epoch 11/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35322.1523 - val_l

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1713.5073 - val_loss: 28579.5332
Epoch 66/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1732.1022 - val_loss: 28082.7559
Epoch 67/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1701.6721 - val_loss: 28957.1582
Epoch 68/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1676.3444 - val_loss: 30236.6055
Epoch 69/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1700.9669 - val_loss: 33273.8086
Epoch 70/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1848.9203 - val_loss: 39935.5000
Epoch 71/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2307.0747 - val_loss: 44039.8945
Epoch 72/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4332.6426 - val_loss: 30929.0215
Epoch 73/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4935.3740 - val_loss: 31138.7031
Epoch 74/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3606.4263 - val_loss: 33914.9219
Epoch 75/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3733.6799 - val_loss: 3

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3569.9548 - val_loss: 26686.5098
Epoch 130/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4601.2285 - val_loss: 24993.4336
Epoch 131/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6190.3643 - val_loss: 30076.3145
Epoch 132/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6429.5742 - val_loss: 50094.0781
Epoch 133/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7988.1104 - val_loss: 37955.3008
Epoch 134/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6525.3242 - val_loss: 30330.4258
Epoch 135/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2917.2727 - val_loss: 29996.1875
Epoch 136/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1906.1849 - val_loss: 31793.0254
Epoch 137/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1452.0564 - val_loss: 35297.4023
Epoch 138/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1835.9625 - val_loss: 36498.3477
Epoch 139/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2165.1123 - v

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1109.3854 - val_loss: 21695.2480
Epoch 194/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1247.7832 - val_loss: 23427.8281
Epoch 195/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1221.3135 - val_loss: 28086.3066
Epoch 196/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1260.3103 - val_loss: 34637.2969
Epoch 197/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1932.2942 - val_loss: 35353.6797
Epoch 198/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2770.7346 - val_loss: 29335.5000
Epoch 199/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2846.0520 - val_loss: 24131.7109
Epoch 200/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3169.2349 - val_loss: 23918.9082
Epoch 201/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3568.3386 - val_loss: 23297.7520
Epoch 202/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2788.1670 - val_loss: 23836.9453
Epoch 203/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1632.9229 - v

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 792.2971 - val_loss: 21735.0234
Epoch 258/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 931.0944 - val_loss: 21949.9004
Epoch 259/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1102.8223 - val_loss: 21723.2480
Epoch 260/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1032.6083 - val_loss: 22112.4551
Epoch 261/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 874.2872 - val_loss: 23049.0859
Epoch 262/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 974.3273 - val_loss: 21857.8340
Epoch 263/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 974.6808 - val_loss: 20979.2910
Epoch 264/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1166.9321 - val_loss: 22088.8516
Epoch 265/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1463.0028 - val_loss: 21875.1855
Epoch 266/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1428.2666 - val_loss: 21573.8496
Epoch 267/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1028.4559 - val_lo

In [28]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    133.387985  119.793729
1    178.025772  359.563884
2    265.367035  168.934525
3    269.071442  355.505368
4    377.038544  433.151080
..          ...         ...
112  450.674774  487.957347
113  154.854019  216.123044
114  271.453644  425.953734
115  200.863052  436.229287
116  325.187256  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 114.86935469408411
Mean Squared Error (MSE): 24730.45771778795
Root Mean Squared Error (RMSE): 157.2592055104818
R-squared (R²): 0.562204202408585
Mean Absolute Error: 114.87 degrees.
Accuracy: 69.06 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
# build model
model3 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model3.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model3.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1)

# evaluate the model
test_loss = model3.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model3.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 233286.2656 - val_loss: 231010.6094
Epoch 2/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 199806.1406 - val_loss: 125850.7031
Epoch 3/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 123208.5469 - val_loss: 79940.7734
Epoch 4/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 93454.2656 - val_loss: 63843.8320
Epoch 5/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 81315.0625 - val_loss: 55589.7109
Epoch 6/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 71951.0469 - val_loss: 49085.0859
Epoch 7/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 64204.1797 - val_loss: 43858.6133
Epoch 8/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 57530.0664 - val_loss: 40100.3906
Epoch 9/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 52091.2812 - val_loss: 37106.6406
Epoch 10/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 47529.0117 - val_loss: 34901.8398
Epoch 11/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43722.7305 - val

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5887.9678 - val_loss: 18454.8359
Epoch 66/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5740.8955 - val_loss: 18370.1953
Epoch 67/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5567.0542 - val_loss: 18361.7441
Epoch 68/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5425.2544 - val_loss: 18280.9688
Epoch 69/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5278.3296 - val_loss: 18233.0664
Epoch 70/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5134.5537 - val_loss: 18170.5938
Epoch 71/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4991.5967 - val_loss: 18112.3184
Epoch 72/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4853.5659 - val_loss: 18064.2422
Epoch 73/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4715.8896 - val_loss: 18005.4883
Epoch 74/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4585.6050 - val_loss: 17942.8965
Epoch 75/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4444.1948 - val_loss: 1

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4394.8833 - val_loss: 18058.1895
Epoch 130/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5023.8584 - val_loss: 20521.7617
Epoch 131/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3776.0562 - val_loss: 22661.8809
Epoch 132/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2607.5149 - val_loss: 21008.7832
Epoch 133/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2247.8738 - val_loss: 19830.4785
Epoch 134/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2007.7708 - val_loss: 19276.6367
Epoch 135/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1860.4501 - val_loss: 18978.3262
Epoch 136/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1782.0295 - val_loss: 18662.4766
Epoch 137/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1729.3063 - val_loss: 18345.7344
Epoch 138/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1706.5372 - val_loss: 18054.0645
Epoch 139/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1703.1543 - v

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3309.3184 - val_loss: 25043.8438
Epoch 194/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3223.4348 - val_loss: 22069.3789
Epoch 195/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2866.5530 - val_loss: 22004.8164
Epoch 196/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2479.4756 - val_loss: 24527.1152
Epoch 197/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2131.1650 - val_loss: 27490.7812
Epoch 198/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2440.6887 - val_loss: 28784.2227
Epoch 199/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2552.5789 - val_loss: 28132.6484
Epoch 200/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2621.4282 - val_loss: 26694.9961
Epoch 201/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2113.4023 - val_loss: 25863.9219
Epoch 202/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1796.2473 - val_loss: 26852.3242
Epoch 203/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1617.2922 - v

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 270.2837 - val_loss: 13993.4902
Epoch 258/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 260.0603 - val_loss: 13975.1699
Epoch 259/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 249.9916 - val_loss: 13957.7158
Epoch 260/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 240.1508 - val_loss: 13940.7295
Epoch 261/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 230.5916 - val_loss: 13924.0010
Epoch 262/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 221.3705 - val_loss: 13907.5996
Epoch 263/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 212.5366 - val_loss: 13891.7959
Epoch 264/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 204.1090 - val_loss: 13877.0127
Epoch 265/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 196.0732 - val_loss: 13863.7002
Epoch 266/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 188.3815 - val_loss: 13852.2949
Epoch 267/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 180.9632 - val_loss: 13

Epoch 322/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 816.3022 - val_loss: 15785.0879
Epoch 323/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 620.6137 - val_loss: 14619.8066
Epoch 324/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 558.6369 - val_loss: 14458.5918
Epoch 325/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 627.5615 - val_loss: 14666.8047
Epoch 326/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 683.2332 - val_loss: 14918.5859
Epoch 327/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 640.7035 - val_loss: 14667.9814
Epoch 328/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 555.9774 - val_loss: 14409.7666
Epoch 329/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 549.0024 - val_loss: 14644.1211
Epoch 330/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 645.9249 - val_loss: 14986.1172
Epoch 331/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 755.2106 - val_loss: 15111.0547
Epoch 332/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 621.1020 

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2837.1191 - val_loss: 20400.0137
Epoch 387/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2588.7292 - val_loss: 18719.6074
Epoch 388/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2331.4102 - val_loss: 17349.6836
Epoch 389/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2282.9956 - val_loss: 16978.9180
Epoch 390/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2362.0784 - val_loss: 17184.0879
Epoch 391/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2280.4294 - val_loss: 17647.1406
Epoch 392/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2186.1279 - val_loss: 17670.5957
Epoch 393/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1967.9087 - val_loss: 17315.2031
Epoch 394/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1693.2528 - val_loss: 16760.1875
Epoch 395/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1315.5779 - val_loss: 16059.2920
Epoch 396/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 941.2244 - va

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1686.3444 - val_loss: 22049.7383
Epoch 451/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1600.6140 - val_loss: 19876.5156
Epoch 452/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1689.0873 - val_loss: 19177.0195
Epoch 453/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1864.6219 - val_loss: 18807.4863
Epoch 454/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2102.1104 - val_loss: 17508.8867
Epoch 455/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1828.6508 - val_loss: 18423.8477
Epoch 456/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1920.3705 - val_loss: 19603.1055
Epoch 457/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2237.1226 - val_loss: 19817.9922
Epoch 458/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2197.8757 - val_loss: 19384.0293
Epoch 459/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2045.8542 - val_loss: 17874.6172
Epoch 460/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1699.2281 - v

In [12]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    -22.091305  119.793729
1    316.375488  359.563884
2     69.236694  168.934525
3    307.607391  355.505368
4    417.406982  433.151080
..          ...         ...
112  624.018188  487.957347
113  184.163910  216.123044
114  379.025055  425.953734
115  373.555359  436.229287
116  323.459534  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 88.68147324237714
Mean Squared Error (MSE): 14127.345956487157
Root Mean Squared Error (RMSE): 118.85851234340416
R-squared (R²): 0.7499078762937121
Mean Absolute Error: 88.68 degrees.
Accuracy: 68.94 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [13]:
# build model
model4 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model4.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model4.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1)

# evaluate the model
test_loss = model4.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model4.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 232477.7500 - val_loss: 210749.7969
Epoch 2/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 165983.0312 - val_loss: 88114.3516
Epoch 3/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 97896.0234 - val_loss: 61681.2266
Epoch 4/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 78993.1016 - val_loss: 49902.3945
Epoch 5/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 66158.7969 - val_loss: 41753.7070
Epoch 6/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 55437.8633 - val_loss: 36115.9531
Epoch 7/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 46325.0391 - val_loss: 32535.9082
Epoch 8/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39703.0117 - val_loss: 30224.7422
Epoch 9/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35027.3750 - val_loss: 28490.9746
Epoch 10/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31513.0977 - val_loss: 26897.1426
Epoch 11/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28654.7305 - val_l

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2247.5366 - val_loss: 22790.1719
Epoch 66/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2210.3757 - val_loss: 23439.3887
Epoch 67/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2183.8987 - val_loss: 24149.3281
Epoch 68/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2171.1736 - val_loss: 24882.5586
Epoch 69/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2182.6506 - val_loss: 25601.1250
Epoch 70/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2224.3247 - val_loss: 26226.7266
Epoch 71/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2307.8821 - val_loss: 26661.2344
Epoch 72/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2442.9019 - val_loss: 26742.7363
Epoch 73/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2638.8933 - val_loss: 26197.9941
Epoch 74/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2888.9495 - val_loss: 24710.9102
Epoch 75/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3173.8682 - val_loss: 2

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1077.7839 - val_loss: 29903.6836
Epoch 130/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1199.7622 - val_loss: 29122.4688
Epoch 131/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1225.9634 - val_loss: 29626.3145
Epoch 132/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1075.1527 - val_loss: 31515.5137
Epoch 133/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1036.6509 - val_loss: 32664.7988
Epoch 134/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1028.3851 - val_loss: 32324.5664
Epoch 135/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1023.1894 - val_loss: 31412.7461
Epoch 136/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1035.7924 - val_loss: 30963.2715
Epoch 137/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1101.5895 - val_loss: 31245.8027
Epoch 138/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1140.8761 - val_loss: 31390.7773
Epoch 139/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1156.3558 - v

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 139.5012 - val_loss: 20551.7285
Epoch 194/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 134.1320 - val_loss: 20560.0117
Epoch 195/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 129.1538 - val_loss: 20568.9453
Epoch 196/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 124.5377 - val_loss: 20578.3672
Epoch 197/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 120.2577 - val_loss: 20588.2148
Epoch 198/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 116.2907 - val_loss: 20598.3750
Epoch 199/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 112.6163 - val_loss: 20608.8008
Epoch 200/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 109.2143 - val_loss: 20619.3809
Epoch 201/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 106.0677 - val_loss: 20630.0762
Epoch 202/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 103.1592 - val_loss: 20640.8008
Epoch 203/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 100.4731 - val_loss: 20

Epoch 258/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 514.9827 - val_loss: 20455.0762
Epoch 259/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 413.8539 - val_loss: 20014.7891
Epoch 260/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 471.2557 - val_loss: 19128.8320
Epoch 261/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 453.4944 - val_loss: 18327.3750
Epoch 262/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 427.0554 - val_loss: 18971.5684
Epoch 263/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 382.7434 - val_loss: 19182.0547
Epoch 264/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 445.6981 - val_loss: 18584.7539
Epoch 265/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 432.8246 - val_loss: 18118.4863
Epoch 266/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 395.6342 - val_loss: 18494.3320
Epoch 267/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 356.2158 - val_loss: 18784.5859
Epoch 268/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 365.1560 

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 271.0090 - val_loss: 19960.5273
Epoch 323/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 196.5388 - val_loss: 20211.1152
Epoch 324/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 139.5071 - val_loss: 20597.6816
Epoch 325/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 113.3631 - val_loss: 21019.8145
Epoch 326/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 104.2189 - val_loss: 21352.1035
Epoch 327/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 96.6077 - val_loss: 21493.0352
Epoch 328/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 84.7029 - val_loss: 21398.9648
Epoch 329/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 70.8289 - val_loss: 21110.2676
Epoch 330/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 61.6562 - val_loss: 20730.4805
Epoch 331/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 64.3239 - val_loss: 20380.7090
Epoch 332/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 80.1183 - val_loss: 20133.55

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 351.0479 - val_loss: 22416.7539
Epoch 387/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 463.2668 - val_loss: 23299.2559
Epoch 388/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 422.4065 - val_loss: 23183.1758
Epoch 389/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 325.6477 - val_loss: 21931.8262
Epoch 390/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 250.7982 - val_loss: 20728.2148
Epoch 391/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 280.4029 - val_loss: 20117.1914
Epoch 392/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 414.9248 - val_loss: 20110.3281
Epoch 393/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 469.7742 - val_loss: 20001.1855
Epoch 394/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 387.4482 - val_loss: 19900.1367
Epoch 395/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 247.0404 - val_loss: 20367.7461
Epoch 396/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 229.0891 - val_loss: 21

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 705.6354 - val_loss: 19966.5723
Epoch 451/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 434.4589 - val_loss: 22009.5293
Epoch 452/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 632.5079 - val_loss: 24275.2793
Epoch 453/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 862.0442 - val_loss: 25129.8105
Epoch 454/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 744.4001 - val_loss: 23507.0195
Epoch 455/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 527.5005 - val_loss: 21675.3613
Epoch 456/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 518.9626 - val_loss: 20635.8867
Epoch 457/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 638.7640 - val_loss: 20441.3262
Epoch 458/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 926.1365 - val_loss: 20408.9316
Epoch 459/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 910.3116 - val_loss: 19978.8594
Epoch 460/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 575.2430 - val_loss: 20

In [14]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    117.657990  119.793729
1    307.476593  359.563884
2    176.979858  168.934525
3    323.755402  355.505368
4    562.170898  433.151080
..          ...         ...
112  672.826965  487.957347
113  267.053589  216.123044
114  424.328156  425.953734
115  325.644714  436.229287
116  363.329376  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 81.40893257367766
Mean Squared Error (MSE): 11547.947585912572
Root Mean Squared Error (RMSE): 107.46137718228151
R-squared (R²): 0.7955701838756482
Mean Absolute Error: 81.41 degrees.
Accuracy: 70.05 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
# build model
model5 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model5.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model5.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.1)

# evaluate the model
test_loss = model5.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model5.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 233117.4219 - val_loss: 220938.5000
Epoch 2/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 176813.5938 - val_loss: 95025.6719
Epoch 3/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 99684.8359 - val_loss: 61497.5898
Epoch 4/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 78049.4844 - val_loss: 50135.5039
Epoch 5/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 64983.6172 - val_loss: 41839.1758
Epoch 6/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 55364.0859 - val_loss: 36423.4141
Epoch 7/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 47716.0117 - val_loss: 32909.7695
Epoch 8/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 41931.3672 - val_loss: 30550.8379
Epoch 9/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37596.3945 - val_loss: 28984.5234
Epoch 10/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33985.8008 - val_loss: 27609.7598
Epoch 11/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30928.73

Epoch 65/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2018.3330 - val_loss: 19839.7812
Epoch 66/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1928.5917 - val_loss: 19548.9844
Epoch 67/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1854.6343 - val_loss: 19474.5859
Epoch 68/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1760.1024 - val_loss: 19120.4570
Epoch 69/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1705.4655 - val_loss: 19051.6426
Epoch 70/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1608.7689 - val_loss: 18678.5234
Epoch 71/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1569.9347 - val_loss: 18564.4316
Epoch 72/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1485.1554 - val_loss: 18261.8320
Epoch 73/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1452.5994 - val_loss: 18072.6055
Epoch 74/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1402.8756 - val_loss: 17864.0957
Epoch 75/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

Epoch 129/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 696.5562 - val_loss: 24091.0898
Epoch 130/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 930.9009 - val_loss: 25463.6250
Epoch 131/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1261.3827 - val_loss: 25267.3457
Epoch 132/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1509.9170 - val_loss: 22842.1074
Epoch 133/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1471.9375 - val_loss: 19762.3203
Epoch 134/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1322.8215 - val_loss: 17778.5488
Epoch 135/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1428.7522 - val_loss: 16885.3262
Epoch 136/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1806.6061 - val_loss: 17567.9336
Epoch 137/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1879.4739 - val_loss: 24841.3867
Epoch 138/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1659.4481 - val_loss: 30470.1523
Epoch 139/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

...

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 430.6245 - val_loss: 20491.6367
Epoch 706/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 577.1448 - val_loss: 19500.3418
Epoch 707/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 396.1110 - val_loss: 18502.1230
Epoch 708/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 332.1430 - val_loss: 18289.7520
Epoch 709/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 575.6251 - val_loss: 20670.6484
Epoch 710/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 636.4099 - val_loss: 20144.5742
Epoch 711/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 416.1744 - val_loss: 20125.2246
Epoch 712/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 343.5309 - val_loss: 20180.8086
Epoch 713/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 429.4527 - val_loss: 20791.8379
Epoch 714/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 601.0485 - val_loss: 19291.8105
Epoch 715/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 427.8885 - va

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 363.3765 - val_loss: 19995.1836
Epoch 770/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 454.1616 - val_loss: 20979.8691
Epoch 771/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 466.2752 - val_loss: 19846.9062
Epoch 772/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 276.3941 - val_loss: 18959.5586
Epoch 773/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 407.0620 - val_loss: 19851.2695
Epoch 774/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 643.7786 - val_loss: 19625.8125
Epoch 775/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 453.2333 - val_loss: 19860.8750
Epoch 776/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 342.4547 - val_loss: 19671.4355
Epoch 777/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 317.4376 - val_loss: 20110.2383
Epoch 778/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 407.8463 - val_loss: 20841.1992
Epoch 779/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 433.3827 - va

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 363.4949 - val_loss: 19948.1289
Epoch 834/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 390.4988 - val_loss: 20153.5664
Epoch 835/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 396.3140 - val_loss: 20691.2695
Epoch 836/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 485.9050 - val_loss: 19175.7734
Epoch 837/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 296.0912 - val_loss: 19052.6797
Epoch 838/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 417.6755 - val_loss: 19419.0938
Epoch 839/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 567.5988 - val_loss: 20009.5742
Epoch 840/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 401.8660 - val_loss: 19524.0664
Epoch 841/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 317.9795 - val_loss: 19904.8438
Epoch 842/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 401.8725 - val_loss: 19801.3750
Epoch 843/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 329.1130 - va

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 280.5044 - val_loss: 17932.0977
Epoch 898/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 242.3602 - val_loss: 18360.4062
Epoch 899/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 312.2540 - val_loss: 18618.4375
Epoch 900/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 381.0134 - val_loss: 18867.3711
Epoch 901/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 324.2961 - val_loss: 17725.8867
Epoch 902/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 234.8012 - val_loss: 17096.5820
Epoch 903/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 291.1716 - val_loss: 17806.3457
Epoch 904/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 632.6518 - val_loss: 18131.9727
Epoch 905/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 334.5488 - val_loss: 17905.1016
Epoch 906/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 282.8945 - val_loss: 18123.9570
Epoch 907/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 323.2025 - va

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 436.4371 - val_loss: 17096.9043
Epoch 962/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 383.1560 - val_loss: 17313.1641
Epoch 963/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 410.6659 - val_loss: 17149.0391
Epoch 964/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 371.8069 - val_loss: 16637.3047
Epoch 965/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 254.1333 - val_loss: 15943.7227
Epoch 966/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 286.4955 - val_loss: 16711.2793
Epoch 967/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 567.8589 - val_loss: 16587.7695
Epoch 968/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 459.5481 - val_loss: 16701.2773
Epoch 969/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 394.5602 - val_loss: 16818.4238
Epoch 970/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 449.3984 - val_loss: 16762.9102
Epoch 971/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 372.9188 - va

In [16]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    162.953720  119.793729
1    293.268066  359.563884
2    177.727203  168.934525
3    393.595215  355.505368
4    423.624695  433.151080
..          ...         ...
112  571.836975  487.957347
113  306.866455  216.123044
114  407.993835  425.953734
115  286.959167  436.229287
116  413.225586  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 73.48076679058765
Mean Squared Error (MSE): 11498.017521044258
Root Mean Squared Error (RMSE): 107.2288091934451
R-squared (R²): 0.7964540806810474
Mean Absolute Error: 73.48 degrees.
Accuracy: 77.22 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [31]:
# build model
model6 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model6.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model6.fit(X_train, y_train, epochs=250, batch_size=32, validation_split=0.1)

# evaluate the model
test_loss = model6.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model6.predict(X_test)

Epoch 1/250


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 233085.7031 - val_loss: 218796.7812
Epoch 2/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 171219.2344 - val_loss: 82199.4531
Epoch 3/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 96444.3203 - val_loss: 59017.1953
Epoch 4/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 78018.1172 - val_loss: 47221.0039
Epoch 5/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 65393.5938 - val_loss: 39566.5430
Epoch 6/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 54863.6328 - val_loss: 34570.8047
Epoch 7/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 47113.3398 - val_loss: 31715.7285
Epoch 8/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 41196.3555 - val_loss: 29997.4922
Epoch 9/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36937.2031 - val_loss: 28670.4512
Epoch 10/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33418.4609 - val_loss: 27721.6387
Epoch 11/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30364.5000 - val_l

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3064.1396 - val_loss: 27004.5488
Epoch 66/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3116.4019 - val_loss: 26004.1582
Epoch 67/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3047.2288 - val_loss: 24064.7070
Epoch 68/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2863.8276 - val_loss: 22255.3711
Epoch 69/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2783.4585 - val_loss: 21225.5352
Epoch 70/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2911.2854 - val_loss: 20928.4258
Epoch 71/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3038.5500 - val_loss: 21391.6367
Epoch 72/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2851.4653 - val_loss: 22820.8320
Epoch 73/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2410.0891 - val_loss: 24630.0449
Epoch 74/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2100.3940 - val_loss: 25844.5078
Epoch 75/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2017.5374 - val_loss: 2

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3192.7788 - val_loss: 34055.8164
Epoch 130/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4145.9907 - val_loss: 33048.8281
Epoch 131/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3190.6997 - val_loss: 29145.1992
Epoch 132/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2858.8677 - val_loss: 30022.2969
Epoch 133/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1930.3640 - val_loss: 28756.0820
Epoch 134/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1670.8029 - val_loss: 30370.6758
Epoch 135/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1314.7407 - val_loss: 30512.2480
Epoch 136/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1139.0844 - val_loss: 32077.5664
Epoch 137/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 988.8801 - val_loss: 33096.8125
Epoch 138/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 917.1353 - val_loss: 34832.5352
Epoch 139/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 922.8134 - val_

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 220.0662 - val_loss: 20771.7012
Epoch 194/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 201.8978 - val_loss: 20040.5801
Epoch 195/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 189.3153 - val_loss: 20757.1562
Epoch 196/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 176.5636 - val_loss: 20127.0664
Epoch 197/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 165.4605 - val_loss: 20732.8867
Epoch 198/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 156.6754 - val_loss: 20180.6777
Epoch 199/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 146.9299 - val_loss: 20702.7578
Epoch 200/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 141.0621 - val_loss: 20210.1641
Epoch 201/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 132.5302 - val_loss: 20669.7988
Epoch 202/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 128.8229 - val_loss: 20223.9746
Epoch 203/250
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 121.3219 - val_loss: 20

In [32]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    119.849564  119.793729
1    435.575287  359.563884
2    151.199036  168.934525
3    365.272736  355.505368
4    484.931427  433.151080
..          ...         ...
112  578.945618  487.957347
113  286.293488  216.123044
114  398.715027  425.953734
115  317.486542  436.229287
116  406.891388  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 77.98496178547845
Mean Squared Error (MSE): 10825.530038855806
Root Mean Squared Error (RMSE): 104.04580740642943
R-squared (R²): 0.8083589227585627
Mean Absolute Error: 77.98 degrees.
Accuracy: 73.6 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
